In [1]:
import os,sys,inspect
currentdir = "D:/BAMT/plan/big_cluster"
# parentdir = os.path.dirname(currentdir)
parentdir = "D:/BAMT"
sys.path.insert(0,parentdir)

In [2]:
import pandas as pd
from bayesian.train_bn import structure_learning
from preprocess.discretization import get_nodes_type, discretization, code_categories
from scipy import stats
import math
import random
from graph.precision_recall import precision_recall
import time

In [3]:
method = 'LL'
steps = 5

In [4]:
true_net = [['Region', 'wd'],
 ['Region', 'blh'],
 ['Region', 'CVD60'],
 ['Region', 'pm2.5'],
 ['Zone', 'no2'],
 ['Zone', 'o3'],
 ['Zone', 'so2'],
 ['Zone', 'co'],
 ['Zone', 'pm10'],
 ['Type', 'no2'],
 ['Type', 'o3'],
 ['Type', 'so2'],
 ['Type', 'co'],
 ['Type', 'pm10'],
 ['Type', 'pm2.5'],
 ['Year', 't2m'],
 ['Year', 'ws'],
 ['Year', 'wd'],
 ['Year', 'tp'],
 ['Year', 'CVD60'],
 ['Year', 'no2'],
 ['Year', 'o3'],
 ['Year', 'so2'],
 ['Year', 'co'],
 ['Year', 'pm10'],
 ['Year', 'pm2.5'],
 ['Season', 'CVD60'],
 ['Month', 't2m'],
 ['Month', 'ws'],
 ['Month', 'wd'],
 ['Month', 'tp'],
 ['Month', 'blh'],
 ['Month', 'CVD60'],
 ['Day', 'ws'],
 ['Day', 'wd'],
 ['Hour', 't2m'],
 ['Hour', 'tp'],
 ['Hour', 'blh'],
 ['Hour', 'ssr'],
 ['Latitude', 't2m'],
 ['Latitude', 'ws'],
 ['Latitude', 'ssr'],
 ['Longitude', 'ws'],
 ['Longitude', 'blh'],
 ['Longitude', 'ssr'],
 ['Altitude', 'ssr'],
 ['Altitude', 'no2'],
 ['t2m', 'ws'],
 ['t2m', 'blh'],
 ['t2m', 'ssr'],
 ['ws', 'tp'],
 ['ws', 'blh'],
 ['ws', 'ssr'],
 ['wd', 't2m'],
 ['wd', 'ws'],
 ['wd', 'blh'],
 ['wd', 'ssr'],
 ['blh', 'ssr'],
 ['blh', 'no2'],
 ['blh', 'o3'],
 ['ssr', 'tp'],
 ['no2', 'ssr'],
 ['o3', 'ssr'],
 ['o3', 'no2'],
 ['so2', 'ssr'],
 ['co', 'ssr'],
 ['co', 'no2'],
 ['co', 'so2'],
 ['pm10', 'ssr'],
 ['pm10', 'no2'],
 ['pm10', 'pm2.5']]

In [5]:
bamt_net = [['Type', 'Region'], ['Type', 'Zone'], ['Month', 'Zone'], ['Region', 'Zone'], ['Zone', 'Year'], ['Month', 'Year'], ['Region', 'Year'], ['Type', 'Season'], ['Region', 'Season'], ['Type', 'Month'], ['Region', 'Month'], ['Season', 'Month'], ['Zone', 'Day'], ['Hour', 'Day'], ['Year', 'Day'], ['Zone', 'Hour'], ['Month', 'Hour'], ['Year', 'Hour'], ['Zone', 'Latitude'], ['Day', 'Latitude'], ['Hour', 'Latitude'], ['Zone', 'Longitude'], ['Day', 'Longitude'], ['Hour', 'Longitude'], ['Zone', 'Altitude'], ['Day', 'Altitude'], ['Hour', 'Altitude'], ['Zone', 't2m'], ['Day', 't2m'], ['Hour', 't2m'], ['Zone', 'ws'], ['Day', 'ws'], ['Hour', 'ws'], ['Zone', 'wd'], ['Day', 'wd'], ['Hour', 'wd'], ['Zone', 'tp'], ['Day', 'tp'], ['Hour', 'tp'], ['Zone', 'blh'], ['Day', 'blh'], ['Hour', 'blh'], ['Zone', 'ssr'], ['Day', 'ssr'], ['Hour', 'ssr'], ['Zone', 'CVD60'], ['Day', 'CVD60'], ['Hour', 'CVD60'], ['Zone', 'no2'], ['Day', 'no2'], ['Hour', 'no2'], ['Zone', 'o3'], ['Day', 'o3'], ['Hour', 'o3'], ['Zone', 'so2'], ['Day', 'so2'], ['Hour', 'so2'], ['Zone', 'co'], ['Day', 'co'], ['Hour', 'co'], ['Zone', 'pm10'], ['Day', 'pm10'], ['Hour', 'pm10'], ['Zone', 'pm2.5'], ['Day', 'pm2.5'], ['Hour', 'pm2.5']]

In [6]:
coded_net = [['Type', 'Region'],
 ['Type', "('Zone', 'Year', 'Day', 'Hour')_in"],
 ['Region', "('Zone', 'Year', 'Day', 'Hour')_in"],
 ['Latitude', "('Zone', 'Year', 'Day', 'Hour')_in"],
 ["('Month', 't2m')_out", 'Season'],
 ["('Zone', 'Year', 'Day', 'Hour')_out", 'Season'],
 ['Region', 'Season'],
 ['Type', "('Month', 't2m')_in"],
 ["('Zone', 'Year', 'Day', 'Hour')_out", "('Month', 't2m')_in"],
 ['Region', "('Month', 't2m')_in"],
 ['Type', 'Latitude'],
 ['Region', 'Latitude'],
 ["('Month', 't2m')_out", 'Longitude'],
 ["('Zone', 'Year', 'Day', 'Hour')_out", 'Longitude'],
 ['Region', 'Longitude'],
 ["('Month', 't2m')_out", 'Altitude'],
 ["('Zone', 'Year', 'Day', 'Hour')_out", 'Altitude'],
 ['Region', 'Altitude'],
 ["('Month', 't2m')_out", 'ws'],
 ["('Zone', 'Year', 'Day', 'Hour')_out", 'ws'],
 ['Region', 'ws'],
 ["('Month', 't2m')_out", 'wd'],
 ["('Zone', 'Year', 'Day', 'Hour')_out", 'wd'],
 ['Region', 'wd'],
 ["('Month', 't2m')_out", 'tp'],
 ["('Zone', 'Year', 'Day', 'Hour')_out", 'tp'],
 ['Region', 'tp'],
 ["('Month', 't2m')_out", 'blh'],
 ["('Zone', 'Year', 'Day', 'Hour')_out", 'blh'],
 ['Region', 'blh'],
 ["('Month', 't2m')_out", 'ssr'],
 ["('Zone', 'Year', 'Day', 'Hour')_out", 'ssr'],
 ['Region', 'ssr'],
 ["('Month', 't2m')_out", 'CVD60'],
 ["('Zone', 'Year', 'Day', 'Hour')_out", 'CVD60'],
 ['Region', 'CVD60'],
 ["('Month', 't2m')_out", 'no2'],
 ["('Zone', 'Year', 'Day', 'Hour')_out", 'no2'],
 ['Region', 'no2'],
 ["('Month', 't2m')_out", 'o3'],
 ["('Zone', 'Year', 'Day', 'Hour')_out", 'o3'],
 ['Region', 'o3'],
 ["('Month', 't2m')_out", 'so2'],
 ["('Zone', 'Year', 'Day', 'Hour')_out", 'so2'],
 ['Region', 'so2'],
 ["('Month', 't2m')_out", 'co'],
 ["('Zone', 'Year', 'Day', 'Hour')_out", 'co'],
 ['Region', 'co'],
 ["('Month', 't2m')_out", 'pm10'],
 ["('Zone', 'Year', 'Day', 'Hour')_out", 'pm10'],
 ['Region', 'pm10'],
 ["('Month', 't2m')_out", 'pm2.5'],
 ["('Zone', 'Year', 'Day', 'Hour')_out", 'pm2.5'],
 ['Region', 'pm2.5'],
 ['Zone', 'Year'],
 ['Zone', 'Day'],
 ['Hour', 'Day'],
 ['Year', 'Day'],
 ['Zone', 'Hour'],
 ['Year', 'Hour'],
 ["('Zone', 'Year', 'Day', 'Hour')_in", 'Zone'],
 ["('Zone', 'Year', 'Day', 'Hour')_in", 'Year'],
 ["('Zone', 'Year', 'Day', 'Hour')_in", 'Day'],
 ["('Zone', 'Year', 'Day', 'Hour')_in", 'Hour'],
 ['Zone', "('Zone', 'Year', 'Day', 'Hour')_out"],
 ['Year', "('Zone', 'Year', 'Day', 'Hour')_out"],
 ['Day', "('Zone', 'Year', 'Day', 'Hour')_out"],
 ['Hour', "('Zone', 'Year', 'Day', 'Hour')_out"],
 ['t2m', 'Month'],
 ["('Month', 't2m')_in", 'Month'],
 ["('Month', 't2m')_in", 't2m'],
 ['Month', "('Month', 't2m')_out"],
 ['t2m', "('Month', 't2m')_out"]]

In [9]:
result = precision_recall(bamt_net, true_net, 3)
print(result)


{'AP': 0.167, 'AR': 0.155, 'AHP': 0.167, 'AHR': 0.155, 'SHD': 115}


In [13]:
coded_net_post = []
for e in coded_net:
    e0 = e[0]
    e1 = e[1]
    if ('_out' not in e0) and ('_in' not in e0) and ('_out' not in e1) and ('_in' not in e1):
        coded_net_post.append(e)

In [14]:
coded_net_post

[['Type', 'Region'],
 ['Region', 'Season'],
 ['Type', 'Latitude'],
 ['Region', 'Latitude'],
 ['Region', 'Longitude'],
 ['Region', 'Altitude'],
 ['Region', 'ws'],
 ['Region', 'wd'],
 ['Region', 'tp'],
 ['Region', 'blh'],
 ['Region', 'ssr'],
 ['Region', 'CVD60'],
 ['Region', 'no2'],
 ['Region', 'o3'],
 ['Region', 'so2'],
 ['Region', 'co'],
 ['Region', 'pm10'],
 ['Region', 'pm2.5'],
 ['Zone', 'Year'],
 ['Zone', 'Day'],
 ['Hour', 'Day'],
 ['Year', 'Day'],
 ['Zone', 'Hour'],
 ['Year', 'Hour'],
 ['t2m', 'Month']]

In [11]:
result = precision_recall(coded_net, true_net, 3)

In [12]:
print(result)

{'AP': 0.068, 'AR': 0.07, 'AHP': 0.055, 'AHR': 0.056, 'SHD': 135}


In [15]:
result = precision_recall(coded_net_post, true_net, 3)
print(result)

{'AP': 0.2, 'AR': 0.07, 'AHP': 0.16, 'AHR': 0.056, 'SHD': 87}


In [16]:
result = precision_recall(bamt_net, true_net, 3)
print(result)

{'AP': 0.167, 'AR': 0.155, 'AHP': 0.167, 'AHR': 0.155, 'SHD': 115}


In [7]:
df_result = pd.DataFrame(columns = ['Strategy', 'Alg', 'AP', 'AR', 'AHP', 'AHR', 'SHD', 'Time'])

In [8]:
geo = pd.read_csv('../data/mehra.csv')
columns = ['Region', 'Zone', 'Type', 'Year', 'Season', 'Month', 'Day', 'Hour',
       'Latitude', 'Longitude', 'Altitude', 't2m', 'ws', 'wd', 'tp', 'blh',
       'ssr', 'CVD60', 'no2', 'o3', 'so2', 'co', 'pm10', 'pm2.5']
geo = geo[columns]
geo.dropna(inplace=True)
geo.reset_index(inplace=True, drop=True)
geo

FileNotFoundError: [Errno 2] No such file or directory: '../data/mehra.csv'

In [7]:
geo['Year'] = geo['Year'].apply(lambda x: str(x))
geo['Month'] = geo['Month'].apply(lambda x: str(x))
geo['Day'] = geo['Day'].apply(lambda x: str(x))
geo['Hour'] = geo['Hour'].apply(lambda x: str(x))

In [8]:
geo_types = get_nodes_type(geo)
geo_types

{'Region': 'disc',
 'Zone': 'disc',
 'Type': 'disc',
 'Year': 'disc',
 'Season': 'disc',
 'Month': 'disc',
 'Day': 'disc',
 'Hour': 'disc',
 'Latitude': 'cont',
 'Longitude': 'cont',
 'Altitude': 'cont',
 't2m': 'cont',
 'ws': 'cont',
 'wd': 'cont',
 'tp': 'cont',
 'blh': 'cont',
 'ssr': 'cont',
 'CVD60': 'cont',
 'no2': 'cont',
 'o3': 'cont',
 'so2': 'cont',
 'co': 'cont',
 'pm10': 'cont',
 'pm2.5': 'cont'}

In [9]:
colums_for_code = []
columns_for_disc = []
for c in columns:
    if geo_types[c] == 'disc':
        colums_for_code.append(c)
    else:
        columns_for_disc.append(c)

In [10]:
geo_coded, label_coder = code_categories(geo, 'label', colums_for_code)
geo_discrete, coder = discretization(geo_coded, 'equal_frequency', columns_for_disc)

# Hill Climbing

# Дискретная структра

In [11]:
for _ in range(steps):
    random.shuffle(columns)
    geo_coded = geo_coded[columns]
    geo_discrete = geo_discrete[columns]
    start_time = time.time()
    bn_geo = structure_learning(geo_discrete, 'HC', geo_types, method)
    full_time = time.time() - start_time
    result = precision_recall(bn_geo['E'], true_net, 3)
    df_result = df_result.append({'Strategy': 'HC', 'Alg':'D', **result, 'Time': full_time}, ignore_index=True)

# Смешанная структура

In [12]:
for _ in range(steps):
    random.shuffle(columns)
    geo_coded = geo_coded[columns]
    geo_discrete = geo_discrete[columns]
    start_time = time.time()
    bn_geo = structure_learning(geo_coded, 'HC', geo_types, method)
    full_time = time.time() - start_time
    result = precision_recall(bn_geo['E'], true_net, 3)
    df_result = df_result.append({'Strategy': 'HC', 'Alg':'M', **result, 'Time': full_time}, ignore_index=True)

# EVO

# Дискретная стурктура

In [13]:
for _ in range(steps):
    random.shuffle(columns)
    geo_coded = geo_coded[columns]
    geo_discrete = geo_discrete[columns]
    start_time = time.time()
    bn_geo = structure_learning(geo_discrete, 'evo', geo_types, method)
    full_time = time.time() - start_time
    result = precision_recall(bn_geo['E'], true_net, 3)
    df_result = df_result.append({'Strategy': 'evo', 'Alg':'D', **result, 'Time': full_time}, ignore_index=True)

Best metric is -25848.0
Generation num: 0
max_depth: 8, no improvements: 0
spent time: 1.1 min
Best metric is -25858.0
Generation num: 1
max_depth: 8, no improvements: 0
spent time: 2.0 min
Best metric is -26029.0
Generation num: 2
max_depth: 8, no improvements: 0
spent time: 2.7 min
Best metric is -26049.0
Generation num: 3
max_depth: 8, no improvements: 0
spent time: 3.5 min
Best metric is -26127.000000000007
Generation num: 4
max_depth: 8, no improvements: 0
spent time: 4.5 min
Best metric is -57464.0
Generation num: 5
max_depth: 8, no improvements: 0
spent time: 5.3 min
Best metric is -58177.0
Generation num: 6
max_depth: 8, no improvements: 0
spent time: 6.3 min
Best metric is -58335.0
Generation num: 7
max_depth: 8, no improvements: 0
spent time: 7.4 min
Best metric is -58345.000000000015
Generation num: 8
max_depth: 8, no improvements: 0
spent time: 8.5 min
Best metric is -79427.0
Generation num: 9
max_depth: 8, no improvements: 0
spent time: 9.7 min
Best metric is -79442.0
Gene

# Смешанная структура

In [14]:
for _ in range(steps):
    random.shuffle(columns)
    geo_coded = geo_coded[columns]
    geo_discrete = geo_discrete[columns]
    start_time = time.time()
    bn_geo = structure_learning(geo_discrete, 'evo', geo_types, method)
    full_time = time.time() - start_time
    result = precision_recall(bn_geo['E'], true_net, 3)
    df_result = df_result.append({'Strategy': 'evo', 'Alg':'M', **result, 'Time': full_time}, ignore_index=True)

Best metric is -38757.0
Generation num: 0
max_depth: 8, no improvements: 0
Number of mutation attempts exceeded. Please check composer requirements for correctness.
spent time: 1.8 min
Best metric is -38757.0
Generation num: 1
max_depth: 8, no improvements: 1
spent time: 2.3 min
Best metric is -39425.0
Generation num: 2
max_depth: 8, no improvements: 0
spent time: 3.0 min
Best metric is -39698.0
Generation num: 3
max_depth: 8, no improvements: 0
spent time: 3.6 min
Best metric is -39805.0
Generation num: 4
max_depth: 8, no improvements: 0
spent time: 4.4 min
Best metric is -39904.0
Generation num: 5
max_depth: 8, no improvements: 0
spent time: 5.0 min
Best metric is -54816.0
Generation num: 6
max_depth: 8, no improvements: 0
spent time: 5.7 min
Best metric is -89050.0
Generation num: 7
max_depth: 8, no improvements: 0
spent time: 6.4 min
Best metric is -90215.0
Generation num: 8
max_depth: 8, no improvements: 0
spent time: 7.3 min
Best metric is -90251.0
Generation num: 9
max_depth: 8,

In [15]:
df_result

,Strategy,Alg,AP,AR,AHP,AHR,SHD,Time
0,HC,D,0.026,0.014,0.026,0.014,108,90.041033
1,HC,D,0.025,0.014,0.025,0.014,109,95.131748
2,HC,D,0.026,0.014,0.026,0.014,108,103.404758
3,HC,D,0.000,0.000,0.000,0.000,110,95.698865
4,HC,D,0.000,0.000,0.000,0.000,109,100.757453
5,HC,M,0.051,0.028,0.026,0.014,107,455.940761
6,HC,M,0.029,0.014,0.000,0.000,104,460.058681
7,HC,M,0.029,0.014,0.029,0.014,103,426.114646
8,HC,M,0.029,0.014,0.029,0.014,103,420.443287
9,HC,M,0.029,0.014,0.000,0.000,104,417.478684


In [16]:
df_result.to_csv('mehra_results_graph.csv', index=False)